In [1]:
from work.models.decoder import RecurrentFeedbackActivityDetectionNetwork

nb_experiment = 1
batch_size = 256
timesteps = 20 # Entre 16 i 30
epochs = 100
output_size = 201

Using Theano backend.


In [2]:
model = RecurrentFeedbackActivityDetectionNetwork(batch_size, timesteps, summary=True)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

____________________________________________________________________________________________________
Layer (type)                       Output Shape        Param #     Connected to                     
lstm1 (LSTM)                       (256, 20, 512)      9852928     lstm_input_1[0][0]               
____________________________________________________________________________________________________
lstm2 (LSTM)                       (256, 20, 512)      2099200     lstm1[0][0]                      
____________________________________________________________________________________________________
fc-o (TimeDistributed)             (256, 20, 201)      103113      lstm2[0][0]                      
Total params: 12055241
____________________________________________________________________________________________________
None


In [22]:
from keras.layers import Dense, Input, TimeDistributed, merge, Masking
from keras.layers.recurrent import LSTM
from keras.models import Model, Sequential

input_features = Input(batch_shape=(batch_size, timesteps, 4096,), name='features')
previous_output = Input(batch_shape=(batch_size, timesteps, 202,), name='prev_output')
merging = merge([input_features, previous_output], mode='concat', concat_axis=-1)


In [8]:
lstm1 = LSTM(512, batch_input_shape=(batch_size, timesteps, 4298),
        return_sequences=True, stateful=True, name='lstm1')(merging)
lstm2 = LSTM(512, batch_input_shape=(batch_size, timesteps, 4298),
        return_sequences=True, stateful=True, name='lstm2')(lstm1)
output = TimeDistributed(Dense(201, activation='softmax'), name='fc')(lstm2)
model = Model(input=[input_features, previous_output], output=output)

In [9]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                       Output Shape        Param #     Connected to                     
features (InputLayer)              (256, 20, 4096)     0                                            
____________________________________________________________________________________________________
prev_output (InputLayer)           (256, 20, 202)      0                                            
____________________________________________________________________________________________________
merge_2 (Merge)                    (256, 20, 4298)     0           features[0][0]                   
                                                                   prev_output[0][0]                
____________________________________________________________________________________________________
lstm1 (LSTM)                       (256, 20, 512)      9852928     merge_2[0][0]           

In [2]:
from work.dataset.activitynet import ActivityNetDataset

dataset = ActivityNetDataset(
    videos_path='../dataset/videos.json',
    labels_path='../dataset/labels.txt'
)

In [5]:
import numpy as np

num_annotations = np.array([video.num_frames // 16 for video in dataset.videos])
num_annotations[np.argsort(num_annotations)][:20000]

array([   1,    1,    2, ..., 1414, 1502, 1826])

In [13]:
max(num_annotations)

23

In [26]:
input_features = Input(shape=(4096,), name='features')
input_masked = Masking(mask_value=0.)(input_features)
lstm1 = LSTM(1024, return_sequences=True, name='lstm1')(input_masked)
lstm2 = LSTM(1024, return_sequences=True, name='lstm2')(lstm1)
lstm3 = LSTM(1024, return_sequences=False, name='lstm3')(lstm2)

temporal_prediction = Dense(10, activation='sigmoid')(lstm3)
flag = Dense(5, activation='softmax')(lstm3)
class_prediction = Dense(201, activation='softmax')(lstm3)

model = Model(input=input_features, output=[temporal_prediction, flag, class_prediction])

Exception: Input 0 is incompatible with layer lstm1: expected ndim=3, found ndim=2

In [25]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                       Output Shape        Param #     Connected to                     
features (InputLayer)              (256, 20, 4096)     0                                            
____________________________________________________________________________________________________
masking_1 (Masking)                (256, 20, 4096)     0           features[0][0]                   
____________________________________________________________________________________________________
lstm1 (LSTM)                       (256, 20, 1024)     20975616    masking_1[0][0]                  
____________________________________________________________________________________________________
lstm2 (LSTM)                       (256, 20, 1024)     8392704     lstm1[0][0]                      
___________________________________________________________________________________________

In [29]:
num_annotations = np.array([len(video.video_id) for video in dataset.videos])
num_annotations[np.argsort(num_annotations)][:1000]

array([11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11

In [32]:
from math import ceil
ceil(.1)

1

In [34]:
from pympler import asizeof

In [36]:
array = np.random.random((512,20,4096))

In [37]:
asizeof.asizeof(array)

335544448

In [38]:
data=[array]
asizeof.asizeof(data)

335544520

In [40]:
from guppy import hpy
h = hpy()
h.heap()

ImportError: No module named 'guppy'